# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 1 2022 3:13PM,243200,10,SONSB0001888,"Nextsource Biotechnology, LLC",Released
1,Aug 1 2022 3:07PM,243199,10,8347830,Yusen – 3D Matrix,Released
2,Aug 1 2022 2:39PM,243194,10,0085919770,ISDIN Corporation,Released
3,Aug 1 2022 2:39PM,243194,10,0085919769,ISDIN Corporation,Released
4,Aug 1 2022 2:39PM,243194,10,0085919599,ISDIN Corporation,Released
5,Aug 1 2022 2:39PM,243194,10,0085919597,ISDIN Corporation,Released
6,Aug 1 2022 2:39PM,243194,10,0085919613,ISDIN Corporation,Released
7,Aug 1 2022 2:39PM,243194,10,0085919614,ISDIN Corporation,Released
8,Aug 1 2022 2:39PM,243194,10,0085919615,ISDIN Corporation,Released
9,Aug 1 2022 2:39PM,243194,10,0085919617,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,243193,Executing,1
40,243193,Released,7
41,243194,Released,25
42,243199,Released,1
43,243200,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243192,NaN,NaN,3.0
243193,NaN,1.0,7.0
243194,NaN,NaN,25.0
243199,NaN,NaN,1.0
243200,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0.0,0.0,1.0
243101,0.0,1.0,0.0
243110,0.0,0.0,3.0
243123,0.0,1.0,1.0
243129,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0,0,1
243101,0,1,0
243110,0,0,3
243123,0,1,1
243129,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,0,0,1
1,243101,0,1,0
2,243110,0,0,3
3,243123,0,1,1
4,243129,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,,,1
1,243101,,1,
2,243110,,,3
3,243123,,1,1
4,243129,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 1 2022 3:13PM,243200,10,"Nextsource Biotechnology, LLC"
1,Aug 1 2022 3:07PM,243199,10,Yusen – 3D Matrix
2,Aug 1 2022 2:39PM,243194,10,ISDIN Corporation
27,Aug 1 2022 2:36PM,243193,19,"AdvaGen Pharma, Ltd"
35,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc
38,Aug 1 2022 2:21PM,243191,10,"Herbivore Botanicals, LLC."
39,Aug 1 2022 2:16PM,243188,16,American International Chemical
40,Aug 1 2022 2:15PM,243190,10,"ClearSpec, LLC"
41,Aug 1 2022 2:08PM,243185,10,"Methapharm, Inc."
42,Aug 1 2022 2:04PM,243177,10,"Akron BioProducts, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 1 2022 3:13PM,243200,10,"Nextsource Biotechnology, LLC",,,1
1,Aug 1 2022 3:07PM,243199,10,Yusen – 3D Matrix,,,1
2,Aug 1 2022 2:39PM,243194,10,ISDIN Corporation,,,25
3,Aug 1 2022 2:36PM,243193,19,"AdvaGen Pharma, Ltd",,1,7
4,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc,,,3
5,Aug 1 2022 2:21PM,243191,10,"Herbivore Botanicals, LLC.",,,1
6,Aug 1 2022 2:16PM,243188,16,American International Chemical,,,1
7,Aug 1 2022 2:15PM,243190,10,"ClearSpec, LLC",,,1
8,Aug 1 2022 2:08PM,243185,10,"Methapharm, Inc.",,,1
9,Aug 1 2022 2:04PM,243177,10,"Akron BioProducts, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 3:13PM,243200,10,"Nextsource Biotechnology, LLC",1,,
1,Aug 1 2022 3:07PM,243199,10,Yusen – 3D Matrix,1,,
2,Aug 1 2022 2:39PM,243194,10,ISDIN Corporation,25,,
3,Aug 1 2022 2:36PM,243193,19,"AdvaGen Pharma, Ltd",7,1,
4,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc,3,,
5,Aug 1 2022 2:21PM,243191,10,"Herbivore Botanicals, LLC.",1,,
6,Aug 1 2022 2:16PM,243188,16,American International Chemical,1,,
7,Aug 1 2022 2:15PM,243190,10,"ClearSpec, LLC",1,,
8,Aug 1 2022 2:08PM,243185,10,"Methapharm, Inc.",1,,
9,Aug 1 2022 2:04PM,243177,10,"Akron BioProducts, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 3:13PM,243200,10,"Nextsource Biotechnology, LLC",1,,
1,Aug 1 2022 3:07PM,243199,10,Yusen – 3D Matrix,1,,
2,Aug 1 2022 2:39PM,243194,10,ISDIN Corporation,25,,
3,Aug 1 2022 2:36PM,243193,19,"AdvaGen Pharma, Ltd",7,1,
4,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 3:13PM,243200,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Aug 1 2022 3:07PM,243199,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Aug 1 2022 2:39PM,243194,10,ISDIN Corporation,25.0,NaN,NaN
3,Aug 1 2022 2:36PM,243193,19,"AdvaGen Pharma, Ltd",7.0,1.0,NaN
4,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc,3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 3:13PM,243200,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Aug 1 2022 3:07PM,243199,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Aug 1 2022 2:39PM,243194,10,ISDIN Corporation,25.0,0.0,0.0
3,Aug 1 2022 2:36PM,243193,19,"AdvaGen Pharma, Ltd",7.0,1.0,0.0
4,Aug 1 2022 2:24PM,243192,10,Eye Pharma Inc,3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3404382,84.0,67.0,19.0
15,972627,13.0,32.0,5.0
16,972621,4.0,7.0,0.0
19,1215793,11.0,2.0,0.0
20,972501,58.0,36.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3404382,84.0,67.0,19.0
1,15,972627,13.0,32.0,5.0
2,16,972621,4.0,7.0,0.0
3,19,1215793,11.0,2.0,0.0
4,20,972501,58.0,36.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,84.0,67.0,19.0
1,15,13.0,32.0,5.0
2,16,4.0,7.0,0.0
3,19,11.0,2.0,0.0
4,20,58.0,36.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,84.0
1,15,Released,13.0
2,16,Released,4.0
3,19,Released,11.0
4,20,Released,58.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,19.0,5.0,0.0,0.0,0.0
Executing,67.0,32.0,7.0,2.0,36.0
Released,84.0,13.0,4.0,11.0,58.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,19.0,5.0,0.0,0.0,0.0
1,Executing,67.0,32.0,7.0,2.0,36.0
2,Released,84.0,13.0,4.0,11.0,58.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,19.0,5.0,0.0,0.0,0.0
1,Executing,67.0,32.0,7.0,2.0,36.0
2,Released,84.0,13.0,4.0,11.0,58.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()